# Introduction

In this project, we'll be building a spam filter for SMS messages using the multinomial Naive Bayes algorithm.

Our dataset comes from Tiago A. Almeida and Jose Maria Gomez Hidalgo, and it can be downloaded from the link below. 
> Dataset Link: https://archive.ics.uci.edu/ml/datasets/sms+spam+collection

You can also find out more about their data collection process in the link below:
> Data Collection Explanation Link: http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition

With that, let's jump in and read in our dataset!

In [1]:
# import our needed libraries
import pandas as pd

# read in our dataset
sms_dataset = pd.read_csv("SMSSpamCollection",sep='\t',header=None,names=['Label','SMS'])

# check how many rows and columns are in the dataset
sms_dataset.shape

(5572, 2)

In [2]:
# check the first 5 rows of our dataset
sms_dataset.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


**NOTE**: "ham" in our dataset is the same as "non-spam."

Let's check the percentage of spam and ham messages we have.

In [3]:
sms_dataset["Label"].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

It looks like we have roughly 87% non-spam messages with the rest of the roughly 13% being spam.

# Building our spam filter

Before we get into designing our spam filter, let's figure out how we are going to test our program (otherwise, once we have designed our spam filter and know the ins and outs of how it works, it might be too tempting to come up with a test for how it works).

In order to properly test our spam filter, we'll need use "new" messages. Instead of generating our own or going out and getting new messages, we can take our sms_dataset and leave out a number of messages from being used in our development. This set will comprise our **test set**. We'll use the rest of the messages as our **training set** to train our program to classify messages as spam or non-spam correctly.

## Creating our training set and test set

For our test set, we'll hold off 20% of our data, leaving for our training set 80% of the rest of our dataset. Since the dataset has 5,572 messages, the breakdown is as follows:
- Training set: 4,458 messages (roughly 80% of our dataset)
- Test set: 1,114 messages (roughly 20% of our dataset)

Our ultimate goal is to create a spam filter that accurately classifies a new message as spam or non-spam more than **80%** of the time.

Now, let's create our training and test sets. We'll start by randomizing the entire dataset so that we spread out the spam and non-spam messages out evenly.

In [4]:
# create a randomized version of our dataset
sms_dataset_random = sms_dataset.sample(frac=1,random_state=1)

# from there, we're going to split off our randomized dataset into a training and test set
eighty_percent_of_records = round(len(sms_dataset_random) * .80)
twenty_percent_of_records = len(sms_dataset_random) - eighty_percent_of_records
training_set = sms_dataset_random.head(eighty_percent_of_records).reset_index(drop=True)
test_set = sms_dataset_random.tail(twenty_percent_of_records).reset_index(drop=True)

# validate the length of each dataset is accurate
len(training_set)

4458

In [5]:
# validate the length of each dataset is accurate
len(test_set)

1114

In [6]:
# validate that the proportion of non-spam to spam still matches with the original dataset
training_set["Label"].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [7]:
test_set["Label"].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

Since the percentage of non-spam and spam of our training and test sets match our original, we're good to go forward!

## Data Cleaning - Removing punctuation and making all words lower case.

We'll clean our training and test sets by removing all punctuation first, and then we'll make all words lower case so the same word but in a different case is not considered a different word.

In [8]:
# clean the SMS column to remove punctuation and make words lower case
training_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [9]:
# function to take in a dataset, remove punctuation, and make words lower case
def clean_text(dataset):
    # make a copy of the dataset
    dataset_copy = dataset.copy()
    # remove punctuation from SMS series
    dataset_copy["SMS"] = dataset_copy["SMS"].str.replace('\W',' ')
    # make all words lowercase
    dataset_copy["SMS"] = dataset_copy["SMS"].str.lower()
    # give us back our cleaned dataset!
    return dataset_copy

# test out the above function
test_test = training_set.head()
test_test

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [10]:
test_test_cleaned = clean_text(test_test)
test_test_cleaned

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


In [11]:
# clean our training_set and test_set
training_set_cleaned = clean_text(training_set)
test_set_cleaned = clean_text(test_set)

## Data Cleaning - Building a table of unique words and their frequencies in spam and non-spam

For the next step in our data cleaning process, we want to create a table that shows each message split out to where each unique word in its own column and the data shows the frequency of that word in spam or non-spam.

We'll start by creating a list of all the unique words that occur in the messages of our training set. This will be our vocabulary.

In [12]:
# split the SMS message into a list of words
training_set_cleaned["SMS_split"] = training_set_cleaned["SMS"].str.split()

# look at the first few records to make sure the message split correctly
training_set_cleaned["SMS_split"].head()

0                    [yep, by, the, pretty, sculpture]
1    [yes, princess, are, you, going, to, make, me,...
2                      [welp, apparently, he, retired]
3                                             [havent]
4    [i, forgot, 2, ask, ü, all, smth, there, s, a,...
Name: SMS_split, dtype: object

In [13]:
# now, we'll create our vocabulary list and iterate over these values to add the words into the list
training_set_vocabulary = []
for word_list in training_set_cleaned["SMS_split"]:
    for word in word_list:
        training_set_vocabulary.append(word)
# make our list a set so we don't have any duplicate words
training_set_vocabulary = set(training_set_vocabulary)
# transform the set back to a list
training_set_vocabulary = list(training_set_vocabulary)
# print the list length
print(len(training_set_vocabulary))

7783


With our vocabulary created, we're one step closer to transforming this vocabulary list into a table. Our next step is to create a dictionary that we'll convert to a dataframe.

In [14]:
# initialize a dictionary where each key is a unique word, and each value is a list of the length of the training set, where each element is 0 at first
word_counts_per_sms = {unique_word: [0] * len(training_set_cleaned["SMS_split"]) 
                       for unique_word in training_set_vocabulary}

# loop through our series and count up how many times the word appears in each message
for index, sms in enumerate(training_set_cleaned["SMS_split"]):
    for word in sms:
        word_counts_per_sms[word][index] += 1

# make the dictionary a dataframe
word_counts_per_sms_df = pd.DataFrame(word_counts_per_sms)
word_counts_per_sms_df.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


With our word counts dataframe created, we'll concatenate our original training set dataframe with our word counts dataframe in order to bring back our Label and SMS colums.

In [15]:
training_set_combined = pd.concat([training_set_cleaned,word_counts_per_sms_df],axis=1)

## Using the Naive Bayes algorithm to classify spam/non-spam on our training set

With our training set dataframe built, we can now try out our spam filter.Let's come up with the inputs for Bayes' theorem.

### Spam
P(Spam | words) = P(Spam and words) / P(words)
or
P(Spam | words) = P(words | Spam) * P(Spam) / P(words)

### Non-Spam (Ham)
P(Ham | words) = P(Ham and words) / P(words)
or
P(Ham | words) = P(words | Ham) * P(Ham) / P(words)

To determine if a new message is spam or non-spam, we'll check P(Spam | words) and P(Ham | words), and whichever probability is greater will lead to that label. 

Since we ultimately only care about which probability is greater rather than the specific probabilities, we can ignore dividing by P(words) in both P(Spam | words) and P(Ham | words) like in the below equations:

### Spam (simplified a bit)
P(Spam | words) *is directly proportional to* P(Spam) * P(words | Spam)

### Non-Spam (simplified a bit)
P(Ham | words) *is directly proportional to* P(Ham) * P(words | Ham)

### A tangent where we get into the weeds of Naive Bayes

The thing with using "words" above is that there can be multiple words that could lead our probability equation being blown out like below for a two word message example:

P(Spam | word_1, word_2) *is directly proportional to* P(Spam) * P(word_1, word_2 | Spam)

where P(word_1, word_2 | Spam) can be further broken down into the following:
P(word_1 *intersects* word_2 | Spam)
which *is directly proportional to*:
P(word_1 | word_2 *intersects* Spam) * P(word_2 | Spam)

If we had three words, we'd have the following:
P(word_1 | word_2 *intersects* word_3 *intersects* Spam) * P(word_2 | word_3 *intersects* Spam) * P(word_3 | Spam)

You can see how out of hand this equation can get with longer messages. To make our lives easier, we're going to assume that each word within a message is independent of each other*. Due to that, we can use the following equation:
P(A *intersects* B) = P(A|B)

So with this, our three word example equation above breaks down into the following:
P(word_1, word_2, word_3 | Spam) = 

P(word_1, word_2, word_3 *intersects* Spam) = 

P(word_1 *intersects* word_2 *intersects* word_3 *intersects* Spam) = 

P(word_1 *intersects* Spam) x P(word_2 *intersects* Spam) * P(word_3 *intersects* Spam) = 

P(word_1 | Spam) x P(word_2 | Spam) x P(word_3 | Spam)

### One more thing - additive smoothing

When trying to see if a message is spam, and if one of the words in a message cannot be found in a spam message, you'll run into an issue with there being a 0 number of successful outcomes over the total words in all of the spam messages, and multiplying a 0 / total words in spam messages with the other probabilities will ultimately just result in a probability of 0. That's not helpful!

We'll use additive smoothing as a result to account for these situations. To do this, we'll add a number ("alpha") to the number of times a word appears in a spam message, and we'll divide by the number of words in across spam messages PLUS alpha * our vocabulary (that is, the total number of unique words across spam and non-spam. NOT all the words we know from our Vocabulary Workshop books)/ 

For determining the probability of P("salutations" | Spam) for instance (assuming "salutations" does not appear in a spam message --> only your interesting relative uses that), we would have the following:

Old formula without additive smoothing (assuming "salutations" doesn't appear in a spam message and there are 500 words across all spam messages):
P("salutations" | Spam) = total number of times "salutations" appears in a spam message / total number of words in spam message
or
P("salutations" | Spam) = 0 / 500 = 0

New formula with additive smoothing (assuming "salutations" doesn't appear in a spam message, there are 500 words across all spam messages, our alpha is 1, and our vocabulary is 1000):
P("salutations" | Spam) = (0 + alpha) / (total number of words in spam message + alpha * total unique words across spam and non-spam)
or
P("salutations" | Spam) = (0 + 1) / (500 + 1 * 1000) = 1 / 1500

Now we don't have any issues with 0 probabilities anymore!
Note that we'll apply this to all words, even those that exist in both spam and non-spam, to make sure that we're fair to all of the probabilities.


## Final formulas for our spam filter to determine whether a message is spam or not

Taking the above into account, below are our final formulas for determining whether a message is spam or non-spam:

### Spam
P(Spam | word_1, word_2, ..., word_n) = P(Spam) * P(word_1 | Spam) * P(word_2 | Spam) * ... * P(word_n | Spam)
where n = number of words in a message
and P(word_n | Spam) = (number of times word_n appears in spam messages + "alpha") / (number of total words in Spam messages + "alpha" * vocabulary)

### Non-Spam
P(Ham | word_1, word_2, ..., word_n) = P(Spam) * P(word_1 | Spam) * P(word_2 | Spam) * ... * P(word_n | Spam)
where n = number of words in a message
and P(word_n | Ham) = (number of times word_n appears in non-spam messages + "alpha") / (number of total words in non-spam messages + "alpha" * vocabulary)

***NOTE: In reality, words in a message are dependent on each other. For instance, messages with "WINNER" are more likely to have the word "money" as well. Because of this, this flavor of Bayes' Theorem is called "simple Bayes" or "Naive Bayes." Since we're ultimately just using our probabilities to compare to each other rather than finding the exact probability, Naive Bayes works just fine for our purposes.

## Next steps

Now that we have our formulas for our spam filter to decide whether a message is spam or not, it's time to plug in the right values! We'll figure out the below from our training set:
- P(Spam)
- P(Ham)
- Number of words across all spam messages
- Number of words across all non-spam messages
- Number of unique words across all of our messages (spam and non-spam messages included) - our Vocabulary

In [16]:
# Calculate the probability of spam messages from our training set. 
# NOTE: we did this above too when looking at the distribution of spam and non-spam in our training set
p_spam = training_set_cleaned[training_set_cleaned["Label"]=='spam'].shape[0] / training_set_cleaned.shape[0]
p_ham = training_set_cleaned[training_set_cleaned["Label"]=='ham'].shape[0] / training_set_cleaned.shape[0]

# find the number of words in each message
training_set_cleaned["n_words"] = training_set_cleaned["SMS_split"].apply(len)

# verify the number of words is correct for a few
n_words_spam = training_set_cleaned[training_set_cleaned["Label"]=='spam']["n_words"].sum()
n_words_ham = training_set_cleaned[training_set_cleaned["Label"]=='ham']["n_words"].sum()

# for our vocabulary, use the length of our training set vocabulary above
n_vocabulary = len(training_set_vocabulary)

# for our alpha, we'll use 1
alpha = 1

With our constat values calculated above, we now need to calculate the probability of each word in our training set vocabulary as illustrated below:
- P(word_n | Spam)
- P(word_n | Ham)

In order to calculate these probabilities for each word, we'll use our equations mentioned above (and reproduced below):
- P(word_n | Spam) = (number of times word_n appears in spam messages + "alpha") / (number of total words in Spam messages + "alpha" * vocabulary)
- P(word_n | Ham) = (number of times word_n appears in non-spam messages + "alpha") / (number of total words in non-spam messages + "alpha" * vocabulary)

In [17]:
# Initialize two dictionaries, one for spam words and the other for non-spam/ham words
# each dictionary will have a key-value pair of key being the unique word in the vocabulary and the value being the number of times it appears in spam or non-spam/ham messages
words_prob_in_spam = {}
for key in training_set_vocabulary:
    words_prob_in_spam[key] = 0

words_prob_in_ham = {}
for key in training_set_vocabulary:
    words_prob_in_ham[key] = 0

In [18]:
# isolate the spam and ham messages in the training set into two different DataFrames
training_set_combined_spam = training_set_combined[training_set_combined["Label"]=='spam']
training_set_combined_ham = training_set_combined[training_set_combined["Label"]=='ham']

In [19]:
# iterate over the spam messages and assign the probability that a word appears in a spam message back to our dictionary
for word in words_prob_in_spam:
    total_appearances_in_spam = training_set_combined_spam[word].sum()
    #calculate the probability that the word appears in spam using our equation above
    p_word_given_spam = (total_appearances_in_spam + alpha) / (n_words_spam + n_vocabulary * alpha)
    words_prob_in_spam[word] = p_word_given_spam
    
# do the same for non-spam/ham messages
for word in words_prob_in_ham:
    total_appearances_in_ham = training_set_combined_ham[word].sum()
    #calculate the probability that the word appears in spam using our equation above
    p_word_given_ham = (total_appearances_in_ham + alpha) / (n_words_ham + n_vocabulary * alpha)
    words_prob_in_ham[word] = p_word_given_ham

# Now we can create our spam filter...

We now have all of the values we need to plug into our equations and classify whether a message is spam or non-spam based on the greater probability value between the two! We'll create a function below that does the following:

### Function to determine if a message is spam or non-spam will do the following:
- Take in a new message as an input
- Calculate P(Spam | words_in_message) and P(Ham | words_in_message)
- Compare the two probabilities, and whichever is greater will determine the classification of spam or non-spam
- If the probabilities happen to equal each other, we'll flag these to let us know that a human's help is needed.


In [20]:
# create our function that will classify a message as spam or non-spam
import re
def classify(message):
    
    # clean up the message
    ## remove all punctuation from our message
    message = re.sub('\W',' ', message)
    
    ## make our message lower case
    message = message.lower()
    
    ## split our message into a list of words in the message based on spaces
    message = message.split()
    
    # calculate p_spam_given_message and p_ham_given_message
    ## initialize the two probabilities
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    ## calculate p_spam_given_message or p_ham_given_message if the word is in spam or ham
    for word in message:
        if word in words_prob_in_ham:
            p_ham_given_message *= words_prob_in_ham[word]
        if word in words_prob_in_spam:
            p_spam_given_message *= words_prob_in_spam[word]
        
    
    # print out our probabilities calculated above
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    # determine which probability is greater and show the appropriate message
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Can\'t determine if it\'s spam or ham. You decide!')
        
# test function
spam_message_test = 'WINNER!! HURRY AND CLICK SECRET CODE TO GET THE MONEY: ABC123!'
non_spam_message_test = 'Hey dude, I\'ll be over there in about 20 min. Don\'t start the movie without me!'
classify(spam_message_test)

P(Spam|message): 4.8646003915562355e-32
P(Ham|message): 6.395453196573382e-34
Label: Spam


# Applying our spam filter function to our test set

Now that we've built our spam filter function, let's apply it to our test set. To do this, we'll modify our function a bit to output a label that says "spam" or "ham" (for non-spam), and apply it to each row of our dataset.

In [21]:
# create the function to be used on our test set
def classify_test_set(message):
    
    # clean up the message
    ## remove all punctuation from our message
    message = re.sub('\W',' ', message)
    
    ## make our message lower case
    message = message.lower()
    
    ## split our message into a list of words in the message based on spaces
    message = message.split()
    
    # calculate p_spam_given_message and p_ham_given_message
    ## initialize the two probabilities
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    ## calculate p_spam_given_message or p_ham_given_message if the word is in spam or ham
    for word in message:
        if word in words_prob_in_ham:
            p_ham_given_message *= words_prob_in_ham[word]
        if word in words_prob_in_spam:
            p_spam_given_message *= words_prob_in_spam[word]
        
    # determine which probability is greater and show the appropriate message
    if p_ham_given_message > p_spam_given_message:
        return "ham"
    elif p_ham_given_message < p_spam_given_message:
        return "spam"
    else:
        return "Can\'t determine if it\'s spam or ham. You decide!"

With our function created, let's now apply this function to our test set.

In [22]:
test_set["predicted_spam_or_ham"] = test_set["SMS"].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted_spam_or_ham
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


# Evaluating our spam filter's performance

Now, let's check our accuracy of our spam filter on our test set. We'll check for the percentage accuracy of the "Label" column matching our "predicted_spam_or_ham" column.

In [26]:
# find the accuracy of our spam filter as mentioned above
test_set["accurate?"] = test_set["Label"] == test_set["predicted_spam_or_ham"]
test_set["accurate?"].value_counts(normalize=True)

True     0.987433
False    0.012567
Name: accurate?, dtype: float64

Our spam filter has around is accurate 98.74% of the time! Very impressive!

# Conclusion

Our goal in this project was to create a spam filter for SMS messages that had at least an 80% accuracy rating in classifying SMS messages as either "spam" or "ham" (non-spam). We used a dataset of over 5000 SMS messages that were labeled by a human as spam or ham, and we then split up our dataset into a training set with 80% of the original data and a test set with the other 20% of the data. We then used Naive Bayes to build our function on our training set, and when we applied our function on the test set, we ended up with a 98.74% accurating rating on our test set, which is incredible. Naive Bayes, folks!

Thanks for reading along! Until next time!